In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install tensorflow

In [2]:
import pandas as pd
import numpy as np
import random
import os
import keras

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
! pip install numpy==1.21.6

  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.21.6 which is incompatible.


In [4]:
# 시드 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [3]:
train = pd.read_csv('/content/drive/MyDrive/kiwom/train.csv')
train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


In [ ]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]

    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]

    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    ####### train / test 분리


    train_size = int(len(data_y) * 0.7)
    train_X = np.array(data_X[0 : train_size])
    train_y = np.array(data_y[0 : train_size])

    test_size = len(data_y) - train_size
    test_X = np.array(data_X[train_size : len(data_X)])
    test_y = np.array(data_y[train_size : len(data_y)])

    print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
    print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)

    ## 표준화 + 모델 진행
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'][train_close['종목코드']==code] = train_scaled

    # 변수의 가중치 화
    tc = train_close['5_10평균']

    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 예측치 평균
    #final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    final_return = np.mean(predictions)

    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


final_df

NameError: name 'train' is not defined

In [ ]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    ## 표준화 + 모델 진행
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'] = train_scaled

    # 데이터 전처리
    # 변수 x
    dfx = train_close[['5_10평균']]
    # 정답 y
    dfy = train_close[['종가']]

    X = dfx.values.tolist()
    y = dfy.values.tolist()

    ###  10일 동안의 OHLVC 데이터로 다음 날의 종가를 예측

    window_size = 15

    data_X = []
    data_y = []
    for i in range(len(y) - window_size):
        _X = X[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
        _y = y[i + window_size]     # 다음 날 종가
        data_X.append(_X)
        data_y.append(_y)
    print(_X, "->", _y)


    ####### train / test 분리

    print('전체 데이터의 크기 :', len(data_X), len(data_y))

    train_size = int(len(data_y) * 0.8)
    train_X = np.array(data_X[0 : train_size])
    train_y = np.array(data_y[0 : train_size])

    test_size = len(data_y) - train_size
    test_X = np.array(data_X[train_size : len(data_X)])
    test_y = np.array(data_y[train_size : len(data_y)])

    print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
    print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)

    # 모델 학습


    model = Sequential()
    model.add(LSTM(units=20, activation='relu', return_sequences=True, input_shape=(15, 1)))
    model.add(Dropout(0.1))
    model.add(LSTM(units=20, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=1))
    model.summary()

    # 모델 학습
    predictions =[]
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(train_X, train_y, epochs=10, batch_size=100)

    # optimizer 쓰기
    from keras import optimizers
    #tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)

    predictions.append(model.predict(test_X)) # 예측

    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': predictions}, ignore_index=True)


final_df


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#### 0725 ver --- 이거로 진행
# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()
date = train['일자'].unique()

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드'])
final_df = final_df.set_index(date)


for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    #train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]

    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]

    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    # 학습 데이터프레임
    for i in range(len(train_close)):
        train_close['5_10평균'][i]


    final_df = final_df.append({'종목코드': code, 'final_return': train_close['5_10평균']}, ignore_index=True)
    final_df = final_df.merge([final_df,],on=final_df.index)

final_df

100%|██████████| 2000/2000 [01:59<00:00, 16.69it/s]


,종목코드,final_return
0,A060310,일자 2021-08-24 -144.5 2021-08-25 -120.5 202...
1,A095570,일자 2021-08-24 -101.0 2021-08-25 -121.0 202...
2,A006840,일자 2021-08-24 -1095.0 2021-08-25 -920.0 2...
3,A054620,일자 2021-08-24 -1065.0 2021-08-25 -1235.0 2...
4,A265520,일자 2021-08-24 -1020.0 2021-08-25 -765.0 2...
...,...,...
1995,A189980,일자 2021-08-24 -264.0 2021-08-25 -262.0 202...
1996,A000540,일자 2021-08-24 -94.0 2021-08-25 -74.5 2021-...
1997,A003280,일자 2021-08-24 0.0 2021-08-25 0.0 2021-...
1998,A037440,일자 2021-08-24 -219.0 2021-08-25 -203.0 202...


In [ ]:
final_df.iloc[0,1]

일자
2021-08-24   -144.5
2021-08-25   -120.5
2021-08-26    -76.5
2021-08-27      8.5
2021-08-30     63.0
              ...  
2023-05-23     15.5
2023-05-24     21.0
2023-05-25     17.0
2023-05-26     78.0
2023-05-30    140.5
Name: 5_10평균, Length: 435, dtype: float64

In [ ]:
https://wikidocs.net/173005

# prophet 모델

In [ ]:
!pip install --upgrade plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 30.9 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [ ]:
%pip install prophet

In [ ]:
! python -m pip install Prophet

In [ ]:
!pip install pystan~=2.14
!pip install fbprophet

In [11]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [16]:
## prophet 모델
from prophet import Prophet
import matplotlib.pyplot as plt
import matplotlib
import itertools
#import talib
#import pmdarima as pm

import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA
import warnings
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

#################

train = pd.read_csv('/content/drive/MyDrive/kiwom/train.csv')

# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame()

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()


# fig, axes = plt.subplots(5,5,figsize =(20,20))
# cnt = 0

## 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
#code = "A103840"
    # 20일선 가격의 상승분과 비교
    train_close = train[train['종목코드'] == code][['일자', '종가', '종목코드']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index(np.arange(len(train_close)), inplace=True)
    train_close.columns = ['일자', '종가','종목코드']

    # n일 이동평균선
    train_close["MA5_mean"]=train_close["종가"].rolling(5).mean()
    train_close["MA10_mean"]=train_close["종가"].rolling(10).mean()
    train_close["MA20_mean"]=train_close["종가"].rolling(20).mean()
    train_close["MA60_mean"]=train_close["종가"].rolling(60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    reg = []
    rev = []

    for i in range(len(train_close)):
        if train_close["MA5_mean"].iloc[i] > train_close["MA10_mean"].iloc[i] and train_close["MA10_mean"].iloc[i] > train_close["MA20_mean"].iloc[i] and train_close["MA20_mean"].iloc[i] > train_close["MA60_mean"].iloc[i]:
            reg.append(1)
            rev.append(0)
        elif train_close["MA5_mean"].iloc[i] < train_close["MA10_mean"].iloc[i] and train_close["MA10_mean"].iloc[i] < train_close["MA20_mean"].iloc[i] and train_close["MA20_mean"].iloc[i] < train_close["MA60_mean"].iloc[i]:
            rev.append(1)
            reg.append(0)
        else:
            reg.append(0)
            rev.append(0)

    train_close["regular"] = pd.Series(reg)
    train_close["reverse"] = pd.Series(rev)

    train_close["reg_sum"] = train_close["regular"].rolling(20).sum()
    train_close["rev_sum"] = train_close["reverse"].rolling(20).sum()
    # train_close["MA20-MA60"] = (train_close["MA20_mean"]-train_close["MA60_mean"])/train_close["MA60_mean"]*50

    # 이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    train_close["MA5-MA10"] = (train_close["MA5_mean"]-train_close["MA10_mean"])/train_close["MA10_mean"]*100
    train_close["score"] = (train_close["reg_sum"]-train_close["rev_sum"]+train_close["MA5-MA10"])

    #가중치
    tc = train_close[["일자","score"]]

    #model = prophet()
    future_df = tc.copy()
    future_df.rename(columns={'일자':'ds','score':'y'},inplace=True)

    m = Prophet(seasonality_mode='additive')
    m.fit(future_df)
    future = m.make_future_dataframe(periods=15)
    forecast = m.predict(future_df)
    forecasted = list(np.array(forecast['yhat'])) # 예측 데이터 리스트

    #m = Prophet(seasonality_mode='additive')
    #m.fit(tc)
    #future = model.make_future_dataframe(periods=15)
    #forecast = model.predict(future)

    # 최종 수익률 계산
    #final_return = predictions.iloc[-1]
    final_return = forecasted[-1]

    # 가장 마지막에 있는 것의 배열상태에 따라 순위 매긴다.
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


results_df

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 77%|███████▋  | 1544/2000 [10:23<02:28,  3.07it/s]INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
 77%|███████▋  | 1545/2000 [10:24<02:16,  3.32it/s]INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw_f9g1w/5qprdwrt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmplw_f9g1w/ukrqvgmz.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56327', 'data', 'file=/tmp/tmplw_f9g1w/5qprdwrt.json', 'init=/tmp/tmplw_f9g1w/ukrqvgmz.json', 'output', 'file=/tmp/tmplw_f9g1w/proph

,종목코드,final_return
0,A060310,-3.581417
1,A095570,-10.412231
2,A006840,4.612834
3,A054620,1.076553
4,A265520,8.291549
...,...,...
1995,A189980,-1.306791
1996,A000540,1.548429
1997,A003280,-1.018602
1998,A037440,-2.839801


In [17]:
results_df # prophet 전체 final_retrun

,종목코드,final_return
0,A060310,-3.581417
1,A095570,-10.412231
2,A006840,4.612834
3,A054620,1.076553
4,A265520,8.291549
...,...,...
1995,A189980,-1.306791
1996,A000540,1.548429
1997,A003280,-1.018602
1998,A037440,-2.839801


In [18]:
#results_df['final_return'] = results_df['final_return'].fillna(0)
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df = results_df.sort_values(by='순위')

sample_submission = pd.read_csv('/content/drive/MyDrive/kiwom/sample_submission.csv')
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='right')
baseline_submission.to_csv('/content/drive/MyDrive/kiwom/0726_MA_test_submission.csv', index=False)
results_df

,종목코드,final_return,순위
1590,A009730,21.127395,1
1160,A058970,20.057214,2
641,A025560,19.933555,3
694,A338220,19.844004,4
715,A335890,19.311855,5
...,...,...,...
1978,A192410,-15.406684,1996
1622,A284740,-15.638465,1997
606,A054180,-16.679578,1998
170,A151910,-17.423170,1999
